In [68]:
import os
import pandas as pd
from datetime import datetime

In [69]:
# CONEXIÓN A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()

In [70]:
import sys
sys.path.append('../')
from sqlalchemy import create_engine, MetaData, select, Table
import schedule
import pandas as pd
from datetime import datetime
import time
from config import POSTGRES_UTEA
import math

USER_DB = POSTGRES_UTEA['USER']
PASS_DB = POSTGRES_UTEA['PASSWORD']
HOST_DB = POSTGRES_UTEA['HOST']
PORT_DB = POSTGRES_UTEA['PORT']
NAME_DB = POSTGRES_UTEA['DATABASE']

ENGINE = create_engine(f'postgresql+psycopg://{USER_DB}:{PASS_DB}@{HOST_DB}:{PORT_DB}/{NAME_DB}')

metadata = MetaData()
reporte_tbl = Table("reporte", metadata, autoload_with=ENGINE, schema="datos_iag")
msj_whatsapp_tbl = Table("msj_whatsapp", metadata, autoload_with=ENGINE, schema="notificaciones_wsp")

In [71]:
def crear_mensaje(datos):    
    try:
        with ENGINE.begin() as conn:
            conn.execute(msj_whatsapp_tbl.insert(), datos.to_dict(orient='records'))
        print("✅ Se ha actualizado ")
    except Exception as e:
        print("❌ Error al insertar en tabla MSJ WHASTAPP", e)
    #return df

In [72]:
# Si existe el archivo, intenta cargarlo
if os.path.exists("mycreds.txt"):
    gauth.LoadCredentialsFile("mycreds.txt")

# Si no hay credenciales o son inválidas, forzar nueva autenticación
if gauth.credentials is None or gauth.access_token_expired:
    try:
        gauth.LocalWebserverAuth()
    except:
        print("Error de autenticación. Eliminá 'mycreds.txt' y volvé a intentarlo.")
else:
    gauth.Authorize()

gauth.SaveCredentialsFile("mycreds.txt")
drive = GoogleDrive(gauth)

In [73]:
ruta = r"C:\Users\bismarksr\Desktop\Planos\planos"  # Cambia por tu ruta
contenido = os.listdir(ruta)
print(contenido)

['PLANO_216.pdf', 'PLANO_217.pdf', 'PLANO_256.pdf']


In [14]:
ruta_xlsx = r"C:\Users\bismarksr\Desktop\Planos\lista_props.xlsx"  # Cambia por tu ruta
df = pd.read_excel(ruta_xlsx)
lista_props = list(set(df['COD_PROPIE']))
len(lista_props)

72

In [28]:
def cargar_pdf_to_drive(id_drive, title_file, path_file):
    try:
        # 1. Buscar si ya existe un archivo con el mismo nombre en la misma carpeta
        query = f"'{id_drive}' in parents and title = '{title_file}' and trashed = false"
        file_list = drive.ListFile({'q': query}).GetList()
        print('aqui')
        if file_list:
            # Si ya existe, usamos el primero encontrado
            file = file_list[0]
            print(f"[INFO] El archivo '{title_file}' ya existe. Se reemplazará.")
        else:
            # Si no existe, creamos uno nuevo
            file = drive.CreateFile({
                "mimeType": "application/pdf",
                "parents": [{"kind": "drive#fileLink", "id": id_drive}]
            })
            file['title'] = title_file
            print(f"[INFO] El archivo '{title_file}' no existe. Se creará uno nuevo.")

        # 2. Reemplazar el contenido
        file.SetContentFile(path_file)
        file.Upload()

        # 3. Dar permisos públicos si es nuevo
        if not file_list:
            file.InsertPermission({
                'type': 'anyone',
                'value': 'anyone',
                'role': 'reader'
            })

        print(f"✅[OK] Archivo '{title_file}' subido exitosamente.")
        return file['alternateLink']

    except Exception as e:
        print(f"❌[ERROR] No se pudo subir el archivo '{title_file}': {e}")
        return False

In [35]:
dic_links = {}

In [30]:
for cod_prop in lista_props:
    file = 'PLANO_' + str(cod_prop) + '.pdf'
    title_file = file
    id_drive = '1-4snItApU4HFbMoDTzwUxXqGdv6teb6I'
    path_file = os.path.join(ruta, file)
    url_drive = cargar_pdf_to_drive(id_drive, title_file, path_file)
    dic_links[cod_prop] = url_drive
    if url_drive == False:
        print(f'Error al cargar plano {file}')

aqui
[INFO] El archivo 'PLANO_216.pdf' no existe. Se creará uno nuevo.
✅[OK] Archivo 'PLANO_216.pdf' subido exitosamente.
aqui
[INFO] El archivo 'PLANO_217.pdf' no existe. Se creará uno nuevo.
✅[OK] Archivo 'PLANO_217.pdf' subido exitosamente.
aqui
[INFO] El archivo 'PLANO_256.pdf' ya existe. Se reemplazará.
✅[OK] Archivo 'PLANO_256.pdf' subido exitosamente.


In [74]:
ruta_xlsx_links = r"C:\Users\bismarksr\Desktop\Planos\dic_links.xlsx"  # Cambia por tu ruta
df_links = pd.read_excel(ruta_xlsx_links)
df_links

,COD_PROPIE,nombre,cod_cos,link
0,17,LA ESTACADA--OLGUITA,1,https://drive.google.com/file/d/1ODlcMT8hKnvjF...
1,1696,LOUISIANA--GARCIA FERNANDO,1,https://drive.google.com/file/d/1IvDXZaPSjpcgt...
2,1748,ETORE--AGUILERA OLGA,1,https://drive.google.com/file/d/1a_vwBm7Gxp9_r...
3,1,SONIMA,5,https://drive.google.com/file/d/1NXcOD62kzUiEJ...
4,19,SANTA BARBARA,5,https://drive.google.com/file/d/14EXgF0Gm8pjXT...
...,...,...,...,...
67,590,HACIENDA PETETO,194,https://drive.google.com/file/d/1NupNK_JNhEMlk...
68,13,GUAYABOCHI--AGUILERA,194,https://drive.google.com/file/d/1UJ_TRuOfzzK-7...
69,55,LA CONQUISTA--AGUILERA,194,https://drive.google.com/file/d/1qGQ-8ec1TXUf7...
70,328,NARANJAL 5 ESTRELLAS--AGUILERA JOSE,194,https://drive.google.com/file/d/1OKPDkC2WIk41r...


In [80]:
ruta_xlsx_contactos = r"C:\Documents\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\CONTACTOS_GRUPO2.xlsx"  # Cambia por tu ruta
df = pd.read_excel(ruta_xlsx_contactos)
df

,COD_COS,NOMBRE,NUM
0,1,NICOLAS AGUILERA,75067427
1,5,JUAN MANUEL ROJAS,76607676
2,6,KLAUS FRERKING,77049491
3,7,DOMINGO NAZER,77624793
4,9,SEBASTIAN DONOSO,76632554
5,11,RODRIGO PAZ,77650982
6,17,ZALZER ANTELO LUIS JHONNY,77397105
7,30,PEDRO ETEROVIC,76344582
8,31,GERMAN CARO,70013202
9,33,MILTON HACOB,76058151


In [81]:
df['fecha_registro'] = datetime.now()
df['cod_canero'] = df['NOMBRE']
df['cod_canero'] = df['COD_COS']
df['nombre_canero'] = df['NOMBRE']
df['numero_cel'] = df['NUM']

df['mensaje'] = df.apply(lambda row: f"""Hola *{row['NOMBRE']}* 👋,

Te enviamos desde UTEA un 📍 plano preliminar donde hemos marcados en azul los lotes que seria opciones a renovar para la campaña 2026. Según nuestra estimativa de producción 📊, tal como siempre recomendamos. 📎 Adjunto el plano para que puedas revisarlo.

Recuerda que esta es solo una herramienta de referencia y la decisión final siempre debe tomarse en campo 🚜 juntos con variables operativas y financieras. ¡Cualquier duda, aquí estoy para ayudarte!

Saludos.""", axis=1)

df['enviado'] = False
df['fecha_enviado'] = None
df['numero_contac'] = '591' + df['NUM'].astype(str) + '@s.whatsapp.net'

In [82]:
df = df.drop(['COD_COS', 'NOMBRE', 'NUM'], axis=1)

In [83]:
df.head(5)

,fecha_registro,cod_canero,nombre_canero,numero_cel,mensaje,enviado,fecha_enviado,numero_contac
0,2025-11-12 09:48:23.238662,1,NICOLAS AGUILERA,75067427,"Hola *NICOLAS AGUILERA* 👋,\n\nTe enviamos desd...",False,None,59175067427@s.whatsapp.net
1,2025-11-12 09:48:23.238662,5,JUAN MANUEL ROJAS,76607676,"Hola *JUAN MANUEL ROJAS* 👋,\n\nTe enviamos des...",False,None,59176607676@s.whatsapp.net
2,2025-11-12 09:48:23.238662,6,KLAUS FRERKING,77049491,"Hola *KLAUS FRERKING* 👋,\n\nTe enviamos desde ...",False,None,59177049491@s.whatsapp.net
3,2025-11-12 09:48:23.238662,7,DOMINGO NAZER,77624793,"Hola *DOMINGO NAZER* 👋,\n\nTe enviamos desde U...",False,None,59177624793@s.whatsapp.net
4,2025-11-12 09:48:23.238662,9,SEBASTIAN DONOSO,76632554,"Hola *SEBASTIAN DONOSO* 👋,\n\nTe enviamos desd...",False,None,59176632554@s.whatsapp.net


In [67]:
crear_mensaje(df)

✅ Se ha actualizado 


In [84]:
ruta_xlsx_contactos = r"C:\Documents\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\CONTACTOS_GRUPO3.xlsx"  # Cambia por tu ruta
df = pd.read_excel(ruta_xlsx_contactos)
df

,COD_COS,NOMBRE,NUM,LINK
0,1,LA ESTACADA--OLGUITA,75067427,https://drive.google.com/file/d/1ODlcMT8hKnvjF...
1,1,LOUISIANA--GARCIA FERNANDO,75067427,https://drive.google.com/file/d/1IvDXZaPSjpcgt...
2,1,ETORE--AGUILERA OLGA,75067427,https://drive.google.com/file/d/1a_vwBm7Gxp9_r...
3,5,SONIMA,76607676,https://drive.google.com/file/d/1NXcOD62kzUiEJ...
4,5,SANTA BARBARA,76607676,https://drive.google.com/file/d/14EXgF0Gm8pjXT...
...,...,...,...,...
67,194,HACIENDA PETETO,77010520,https://drive.google.com/file/d/1NupNK_JNhEMlk...
68,194,GUAYABOCHI--AGUILERA,77010520,https://drive.google.com/file/d/1UJ_TRuOfzzK-7...
69,194,LA CONQUISTA--AGUILERA,77010520,https://drive.google.com/file/d/1qGQ-8ec1TXUf7...
70,194,NARANJAL 5 ESTRELLAS--AGUILERA JOSE,77010520,https://drive.google.com/file/d/1OKPDkC2WIk41r...


In [85]:
df['fecha_registro'] = datetime.now()
df['cod_canero'] = df['NOMBRE']
df['cod_canero'] = df['COD_COS']
df['nombre_canero'] = df['NOMBRE']
df['numero_cel'] = df['NUM']

df['mensaje'] = df.apply(lambda row: f"""*PROPIEDAD:* {row['NOMBRE']},
{row['LINK']}
""", axis=1)

df['enviado'] = False
df['fecha_enviado'] = None
df['numero_contac'] = '591' + df['NUM'].astype(str) + '@s.whatsapp.net'

In [87]:
df

,COD_COS,NOMBRE,NUM,LINK,fecha_registro,cod_canero,nombre_canero,numero_cel,mensaje,enviado,fecha_enviado,numero_contac
0,1,LA ESTACADA--OLGUITA,75067427,https://drive.google.com/file/d/1ODlcMT8hKnvjF...,2025-11-12 09:51:08.101850,1,LA ESTACADA--OLGUITA,75067427,"*PROPIEDAD:* LA ESTACADA--OLGUITA,\nhttps://dr...",False,None,59175067427@s.whatsapp.net
1,1,LOUISIANA--GARCIA FERNANDO,75067427,https://drive.google.com/file/d/1IvDXZaPSjpcgt...,2025-11-12 09:51:08.101850,1,LOUISIANA--GARCIA FERNANDO,75067427,"*PROPIEDAD:* LOUISIANA--GARCIA FERNANDO,\nhttp...",False,None,59175067427@s.whatsapp.net
2,1,ETORE--AGUILERA OLGA,75067427,https://drive.google.com/file/d/1a_vwBm7Gxp9_r...,2025-11-12 09:51:08.101850,1,ETORE--AGUILERA OLGA,75067427,"*PROPIEDAD:* ETORE--AGUILERA OLGA,\nhttps://dr...",False,None,59175067427@s.whatsapp.net
3,5,SONIMA,76607676,https://drive.google.com/file/d/1NXcOD62kzUiEJ...,2025-11-12 09:51:08.101850,5,SONIMA,76607676,"*PROPIEDAD:* SONIMA,\nhttps://drive.google.com...",False,None,59176607676@s.whatsapp.net
4,5,SANTA BARBARA,76607676,https://drive.google.com/file/d/14EXgF0Gm8pjXT...,2025-11-12 09:51:08.101850,5,SANTA BARBARA,76607676,"*PROPIEDAD:* SANTA BARBARA,\nhttps://drive.goo...",False,None,59176607676@s.whatsapp.net
...,...,...,...,...,...,...,...,...,...,...,...,...
67,194,HACIENDA PETETO,77010520,https://drive.google.com/file/d/1NupNK_JNhEMlk...,2025-11-12 09:51:08.101850,194,HACIENDA PETETO,77010520,"*PROPIEDAD:* HACIENDA PETETO,\nhttps://drive.g...",False,None,59177010520@s.whatsapp.net
68,194,GUAYABOCHI--AGUILERA,77010520,https://drive.google.com/file/d/1UJ_TRuOfzzK-7...,2025-11-12 09:51:08.101850,194,GUAYABOCHI--AGUILERA,77010520,"*PROPIEDAD:* GUAYABOCHI--AGUILERA,\nhttps://dr...",False,None,59177010520@s.whatsapp.net
69,194,LA CONQUISTA--AGUILERA,77010520,https://drive.google.com/file/d/1qGQ-8ec1TXUf7...,2025-11-12 09:51:08.101850,194,LA CONQUISTA--AGUILERA,77010520,"*PROPIEDAD:* LA CONQUISTA--AGUILERA,\nhttps://...",False,None,59177010520@s.whatsapp.net
70,194,NARANJAL 5 ESTRELLAS--AGUILERA JOSE,77010520,https://drive.google.com/file/d/1OKPDkC2WIk41r...,2025-11-12 09:51:08.101850,194,NARANJAL 5 ESTRELLAS--AGUILERA JOSE,77010520,*PROPIEDAD:* NARANJAL 5 ESTRELLAS--AGUILERA JO...,False,None,59177010520@s.whatsapp.net


In [88]:
crear_mensaje(df)

✅ Se ha actualizado 
